## Testing the electrophysiological properties : Inhibitory/GABAergic neurons

We discuss the properties of three types of GABAergic interneurons - PV expressing, SST expressing and VIP expressing. However, the electrophysiological properties of these are extracted from the two types of neurons called fast spiking (fs) and non-fast spiking (nfs) neurons, from the literature.

In [12]:
%matplotlib notebook

import brian2 as b2
import numpy
import random
import matplotlib.pyplot as plt
import pandas as pd
import datetime

from input_factory import get_step_current, get_ou_current
from plot_tools import plot_voltage_and_current_traces

In [2]:
model_eqs = '''
dv/dt = (-gL*(v-v_rest) - w1 - w2 + I_ext(t))/C : volt

dw1/dt = -w1/tau_w1 : amp
dw2/dt = -w2/tau_w2 : amp

dvt1/dt = -vt1/tau_vt1 : volt
dvt2/dt = -vt2/tau_vt2 : volt

vt = v_thresh + vt1 + vt2 : volt
lambda_t = lambda_0*exp((v-vt)/del_v): Hz
'''
reset_eqs = '''
v = v_reset
w1+=amp_w1
w2+=amp_w2
vt1+=amp_vt1
vt2+=amp_vt2
'''

In [3]:
nfs_df = pd.read_csv('data/nfs.txt',header = None)
nfs_df.columns =['C (nF)','gL (usiemens)','v_rest (mV)','v_reset (mV)','tau_refr (ms)',\
             'v_thresh (mV)','del_v (mV)', 'amp_w1 (nA)','tau_w1 (ms)','amp_w2 (nA)',\
             'tau_w2 (ms)','amp_vt1 (mV)','tau_vt1 (ms)','amp_vt2 (mV)','tau_vt2 (ms)']
fs_df = pd.read_csv('data/fs.txt',header = None)
fs_df.columns =['C (nF)','gL (usiemens)','v_rest (mV)','v_reset (mV)','tau_refr (ms)',\
             'v_thresh (mV)','del_v (mV)', 'amp_w1 (nA)','tau_w1 (ms)','amp_w2 (nA)',\
             'tau_w2 (ms)','amp_vt1 (mV)','tau_vt1 (ms)','amp_vt2 (mV)','tau_vt2 (ms)']

In [4]:
nfs_df

,C (nF),gL (usiemens),v_rest (mV),v_reset (mV),tau_refr (ms),v_thresh (mV),del_v (mV),amp_w1 (nA),tau_w1 (ms),amp_w2 (nA),tau_w2 (ms),amp_vt1 (mV),tau_vt1 (ms),amp_vt2 (mV),tau_vt2 (ms)
0,0.046083,0.006613,-71.197,-48.420,4,-48.085,0.63436,0.031839,11.459,0.001565,500.000,5.6645,11.548,0.63820,473.670
1,0.030942,0.005471,-63.021,-42.268,4,-39.358,0.78475,0.265740,80.066,-0.232230,85.274,4.7549,92.607,-2.36170,92.607
2,0.041975,0.002926,-61.654,-40.247,4,-49.110,1.29580,0.015207,13.165,0.016220,70.760,5.8349,31.895,1.93800,382.210
3,0.045642,0.006588,-71.209,-48.420,4,-48.277,0.64666,0.031817,11.284,0.001486,500.000,5.5064,12.821,0.61197,479.380
4,0.054226,0.005462,-59.786,-38.673,4,-44.469,1.23730,0.047379,19.093,-0.000802,282.060,4.8531,14.142,1.48370,347.690
5,0.038200,0.005937,-67.806,-45.313,4,-41.866,0.75960,0.051750,34.338,-0.012963,118.450,-4.0874,66.752,5.95560,66.752


In [5]:
fs_df

,C (nF),gL (usiemens),v_rest (mV),v_reset (mV),tau_refr (ms),v_thresh (mV),del_v (mV),amp_w1 (nA),tau_w1 (ms),amp_w2 (nA),tau_w2 (ms),amp_vt1 (mV),tau_vt1 (ms),amp_vt2 (mV),tau_vt2 (ms)
0,0.049873,0.005796,-72.135,-56.112,4,-49.064,0.58708,0.045860,17.158,-0.013761,71.309,2.27260,18.146,-1.174800,18.146
1,0.056424,0.006658,-70.121,-60.106,4,-49.251,0.51066,0.738690,40.800,-0.682990,43.726,0.11264,1000.000,0.339650,111.860
2,0.047600,0.013032,-62.712,-53.610,4,-46.067,0.97566,0.067409,11.011,-0.011931,169.550,16.42100,61.974,-16.876000,59.358
3,0.035515,0.006580,-66.289,-51.885,4,-41.730,0.64993,0.048545,20.590,-0.013581,138.650,2.78930,61.094,-1.847800,61.094
4,0.038200,0.005937,-67.806,-45.313,4,-41.619,0.75960,0.051750,34.338,-0.012963,118.450,-4.08740,66.752,5.955600,66.752
5,0.048262,0.010784,-72.042,-56.229,4,-46.529,0.74107,0.101660,22.626,-0.032768,108.840,3.44750,7.197,0.172790,417.120
6,0.050049,0.005742,-72.376,-55.588,4,-49.345,0.58798,0.047068,16.881,-0.014306,68.874,4.21170,70.676,-3.734000,70.676
7,0.056894,0.006680,-70.036,-60.106,4,-49.107,0.49588,0.754400,41.290,-0.697830,44.256,0.40880,176.850,0.001768,176.760
8,0.048262,0.010784,-72.042,-56.229,4,-46.550,0.74107,0.101660,22.626,-0.032768,108.840,3.44750,7.197,0.172790,417.120


In [13]:
nfs_df

,C (nF),gL (usiemens),v_rest (mV),v_reset (mV),tau_refr (ms),v_thresh (mV),del_v (mV),amp_w1 (nA),tau_w1 (ms),amp_w2 (nA),tau_w2 (ms),amp_vt1 (mV),tau_vt1 (ms),amp_vt2 (mV),tau_vt2 (ms)
0,0.046083,0.006613,-71.197,-48.420,4,-48.085,0.63436,0.031839,11.459,0.001565,500.000,5.6645,11.548,0.63820,473.670
1,0.030942,0.005471,-63.021,-42.268,4,-39.358,0.78475,0.265740,80.066,-0.232230,85.274,4.7549,92.607,-2.36170,92.607
2,0.041975,0.002926,-61.654,-40.247,4,-49.110,1.29580,0.015207,13.165,0.016220,70.760,5.8349,31.895,1.93800,382.210
3,0.045642,0.006588,-71.209,-48.420,4,-48.277,0.64666,0.031817,11.284,0.001486,500.000,5.5064,12.821,0.61197,479.380
4,0.054226,0.005462,-59.786,-38.673,4,-44.469,1.23730,0.047379,19.093,-0.000802,282.060,4.8531,14.142,1.48370,347.690
5,0.038200,0.005937,-67.806,-45.313,4,-41.866,0.75960,0.051750,34.338,-0.012963,118.450,-4.0874,66.752,5.95560,66.752


In [24]:
1/fs_df.iloc[1,:][0]

17.722954771019424

For PV positive neurons, we use fast-spiking properties

In [6]:
fs_param = fs_df.iloc[2,:] #set which value to try out, set any if running through all
C = fs_param['C (nF)'] * b2.nF
gL = fs_param['gL (usiemens)'] * b2.usiemens
v_rest = fs_param['v_rest (mV)'] * b2.mV
v_reset = fs_param['v_reset (mV)'] * b2.mV
tau_refr = fs_param['tau_refr (ms)'] * b2.ms

v_thresh = fs_param['v_thresh (mV)'] * b2.mV
del_v = fs_param['del_v (mV)'] * b2.mV

amp_w1 = fs_param['amp_w1 (nA)'] * b2.nA
tau_w1 = fs_param['tau_w1 (ms)'] * b2.ms
amp_w2 = fs_param['amp_w2 (nA)'] * b2.nA
tau_w2 = fs_param['tau_w2 (ms)'] * b2.ms

amp_vt1 = fs_param['amp_vt1 (mV)'] * b2.mV
tau_vt1 = fs_param['tau_vt1 (ms)'] * b2.ms
amp_vt2 = fs_param['amp_vt2 (mV)'] * b2.mV
tau_vt2 = fs_param['tau_vt2 (ms)'] * b2.ms

In [7]:
## amplitude to be played with 
# I_ext = get_step_current(t_start = 200, t_end = 450, unit_time = 1*b2.ms, amplitude = 0.15*b2.namp)
I_ext = get_ou_current(12.1)
lambda_0 = 10 * b2.kHz
time = 1000 * b2.ms

PV = b2.NeuronGroup(1, model = model_eqs, reset = reset_eqs, threshold = "rand() < lambda_t*dt",refractory = tau_refr, method = 'rk4')
PV.v = v_rest
voltage_monitor_pv = b2.StateMonitor(PV, ['v', 'vt','w1','w2'], record=True)
spike_monitor_pv = b2.SpikeMonitor(PV)

pv_monitors = [voltage_monitor_pv, spike_monitor_pv]
print("Before simulation" , datetime.datetime.now())

pv_net = b2.Network(PV)
pv_net.add(pv_monitors)
pv_net.run(time)

print("After simulation" , datetime.datetime.now())

Before simulation 2018-07-20 13:09:33.052579
After simulation 2018-07-20 13:09:41.674536


In [8]:
fig = plt.figure(figsize=(5,10))
plot_voltage_and_current_traces(voltage_monitor_pv, spike_monitor_pv,
                                I_ext)
plt.show()

<IPython.core.display.Javascript object>

In [9]:
fig.savefig('../figs/ou_all.jpg')

# Using NFS properties

For VIP, we use NFS neuron properties

In [ ]:
nfs_param = nfs_df.iloc[0,:] #set which value to try out, set any if running through all
C = nfs_param['C (nF)'] * b2.nF
gL = nfs_param['gL (usiemens)'] * b2.usiemens
v_rest = nfs_param['v_rest (mV)'] * b2.mV
v_reset = nfs_param['v_reset (mV)'] * b2.mV
tau_refr = nfs_param['tau_refr (ms)'] * b2.ms

v_thresh = nfs_param['v_thresh (mV)'] * b2.mV
del_v = nfs_param['del_v (mV)'] * b2.mV

amp_w1 = nfs_param['amp_w1 (nA)'] * b2.nA
tau_w1 = nfs_param['tau_w1 (ms)'] * b2.ms
amp_w2 = nfs_param['amp_w2 (nA)'] * b2.nA
tau_w2 = nfs_param['tau_w2 (ms)'] * b2.ms

amp_vt1 = nfs_param['amp_vt1 (mV)'] * b2.mV
tau_vt1 = nfs_param['tau_vt1 (ms)'] * b2.ms
amp_vt2 = nfs_param['amp_vt2 (mV)'] * b2.mV
tau_vt2 = nfs_param['tau_vt2 (ms)'] * b2.ms

In [ ]:
## amplitude to be played with 
# I_ext = get_step_current(t_start = 100, t_end = 700, unit_time = 1*b2.ms, amplitude = 0.2*b2.namp)
I_ext = get_ou_current(14)
rate_0 = b2.defaultclock.dt/b2.ms * b2.Hz
time = 1000 * b2.ms
lambda_0 = 10 * b2.kHz
VIP = b2.NeuronGroup(1, model = model_eqs, reset = reset_eqs, threshold = "rand() < lambda_t*dt",refractory = tau_refr, method = 'rk4')
VIP.v = v_rest
voltage_monitor_vip = b2.StateMonitor(VIP, ['v','vt','w1','w2'], record=True)
spike_monitor_vip = b2.SpikeMonitor(VIP)

vip_monitors = [voltage_monitor_vip, spike_monitor_vip]
print("Before simulation" , datetime.datetime.now())

vip_net = b2.Network(VIP)
vip_net.add(vip_monitors)
vip_net.run(time)

print("After simulation" , datetime.datetime.now())

Before simulation 2018-07-20 13:09:43.296456


clang: warning: libstdc++ is deprecated; move to libc++ with a minimum deployment target of OS X 10.9 [-Wdeprecated]
clang: warning: libstdc++ is deprecated; move to libc++ with a minimum deployment target of OS X 10.9 [-Wdeprecated]
clang: warning: libstdc++ is deprecated; move to libc++ with a minimum deployment target of OS X 10.9 [-Wdeprecated]
clang: warning: libstdc++ is deprecated; move to libc++ with a minimum deployment target of OS X 10.9 [-Wdeprecated]


KeyboardInterrupt: 

In [ ]:
fig2 = plt.figure(figsize=(5,10))
plot_voltage_and_current_traces(voltage_monitor_vip, spike_monitor_vip, I_ext)
plt.show()

For SST neurons, we know that the C is low-ish (high R_in) and the properties used by me are like nfs.

In [ ]:
fig2.savefig('../figs/nounfs.jpg')